### LB: 0.97018   Private Score: 0.97121

As it was the first competition to work seriously, I will leave it in Kernel as a memorial...

My coding skill is poor.. So, I'm glad to point out the wrong codes.

Thank you.

In [ ]:
import os, gc, time, datetime, hashlib, random, pickle
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification 

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, SGDClassifier, Ridge
from sklearn.svm import SVC, NuSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA, TruncatedSVD, KernelPCA
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.mixture import GaussianMixture
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.feature_selection import VarianceThreshold
from sklearn.cluster import KMeans

import optuna
optuna.logging.disable_default_handler()

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
pd.set_option('max_columns', 9999)
pd.set_option('max_rows', 9999)

start = time.time()
os.listdir('../input/')

In [ ]:
def load_data(nrows=None, verbose=True):
    train = pd.read_csv('../input/train.csv', nrows=nrows)
    test = pd.read_csv('../input/test.csv')
            
    if verbose:
        print('Train Shape: {}'.format(train.shape))
        print('Test Shape: {}'.format(test.shape))
    
    return train, test

In [ ]:
# https://www.kaggle.com/christofhenkel/graphicallasso-gaussianmixture
from sklearn.covariance import GraphicalLasso

def get_mean_cov(x,y):
    model = GraphicalLasso()
    ones = (y==1).astype(bool)
    x2 = x[ones]
    model.fit(x2)
    p1 = model.precision_
    m1 = model.location_
    
    onesb = (y==0).astype(bool)
    x2b = x[onesb]
    model.fit(x2b)
    p2 = model.precision_
    m2 = model.location_
    
    ms = np.stack([m1,m2])
    ps = np.stack([p1,p2])
    return ms,ps

In [ ]:
def trainer(train, test, cv, params, model_type='qda', _scaler=True, SVD=None, 
            variance=False, _model=None, _plot=True, threshold=1.5):
    _train, _test = train.copy(), test.copy()
#     train_org, _ = load_data(verbose=False)
    _test['target'] = -1
    cols = [col for col in _train.columns if not col in ['id', 'target', 'wheezy-copper-turtle-magic']]
    
    # Setting
    preds = np.zeros(len(_test))
    oof = np.zeros(len(_train))
    oof_org = np.zeros(len(train_org))
    
    for i in tqdm(range(512)):
        train_2 = _train[_train['wheezy-copper-turtle-magic'] == i]
        test_2 = _test[_test['wheezy-copper-turtle-magic'] == i]
        train_2_org = train_org[train_org['wheezy-copper-turtle-magic'] == i]
        
        len_train = len(train_2)
        Y = train_2['target'].reset_index(drop=True).values
        train_index = train_2.index
        test_index = test_2.index
        train_org_index = train_2_org.index
        
        # Concat Train/Test Data
        merged_data = pd.concat([train_2, test_2], axis=0).reset_index(drop=True)[cols].values
        
        # Dimension Compression
        if SVD is not None:
            clf = TruncatedSVD(n_components=SVD)
            _X = clf.fit_transform(merged_data)
            X_org = clf.transform(train_2_org[cols].values)
        elif variance:
            clf = VarianceThreshold(threshold=threshold)
            _X = clf.fit_transform(merged_data)
            X_org = clf.transform(train_2_org[cols].values)
        else:
            _X = merged_data
            X_org = clf.transform(train_2_org[cols].values)            
        
        # Scaler
        if _scaler:
            _scaler = StandardScaler()
            _X = _scaler.fit_transform(_X)
            X_org = _scaler.transform(X_org)
        
        # Divide Train, Test data
        X = _X[:len_train]
        test_2 = _X[len_train:]
        del merged_data, _X
        gc.collect()
        
        # Change dtype
        X = X.astype(np.float32)
        test_2 = test_2.astype(np.float32)
        X_org = X_org.astype(np.float32)
        
        # Model
        for i, (trn_index, val_index) in enumerate(cv.split(X, Y)):
            
            if model_type == 'gmm':
                ms, ps = get_mean_cov(X, Y)
            
            X_trn, X_val = X[trn_index], X[val_index]
            Y_trn, Y_val = Y[trn_index], Y[val_index]
            
            if model_type == 'nusvc':
                model = NuSVC(**params)
            elif model_type == 'qda':
                model = QuadraticDiscriminantAnalysis(**params)
            elif model_type == 'gmm':
                model = GaussianMixture(**params, means_init=ms, precisions_init=ps)
            elif model_type == None:
                model = _model
                
            model.fit(X_trn, Y_trn)
            oof[train_index[val_index]] += model.predict_proba(X_val)[:,1]
            preds[test_index] += model.predict_proba(test_2)[:,1] / cv.n_splits
            oof_org[train_org_index] += model.predict_proba(X_org)[:,1] / cv.n_splits
    
    # reverse result only GMM
    if model_type == 'gmm':
        oof = 1.0 - oof
        preds = 1.0 - preds
        oof_org = 1.0 - oof_org
          
    # AUC
    auc = roc_auc_score(_train['target'], oof[:_train.shape[0]])
    # AUC By Original Train Data
    auc_org = roc_auc_score(train_org['target'], oof_org)
    
    # Plotting Histogram
    if _plot:
        fig = plt.figure(figsize=(6, 4))
        sns.distplot(np.array(preds), kde=False, bins=25)
        plt.title('Model [{}]  AUC: {:.5f}'.format(model_type, auc_org))
        plt.show()
    
    return np.array(preds), oof[:train.shape[0]]

In [ ]:
train, test = load_data()
train_org = train.copy()

# Psuedo Labeling

In [ ]:
def PsuedoLabeling(train, test, reg_param, _min=0.01, _max=0.99):
    _train = train.copy()
    _test = test.copy()
    # First Model
    qda_params = {
        'reg_param': reg_param
    }
    cv = StratifiedKFold(n_splits=15)
    preds, oof = trainer(_train, _test, cv, qda_params, model_type='qda', variance=True, _plot=False, _scaler=False)
    
    _test['target'] = preds
    test_0 = _test[_test['target'] < _min]
    test_0['target'] = 0
    test_1 = _test[_test['target'] > _max]
    test_1['target'] = 1
    
    train = pd.concat([train, test_0, test_1], axis=0).reset_index(drop=True)
    
    return train


def run_PsuedoLabeling(train, test, reg_param, _min=0.01, _max=0.99, _iter=20):
    reg_param = random.uniform(0.1, 0.6)
    for _ in range(_iter):
        before_train_shape = train.shape[0]
        train = PsuedoLabeling(train, test, reg_param, _min=_min, _max=_max)
        train = train.drop_duplicates(keep='first').reset_index(drop=True)
        print('Train Data Shape: {}'.format(train.shape))
        if train.shape[0] ==before_train_shape:
            break
            
    return train

In [ ]:
def change_label(train, test, reg_param):
    modified_train = train.copy()
    _train = train.copy()
    _test = test.copy()
    
    qda_params = {
        'reg_param': reg_param
    }
    cv = StratifiedKFold(n_splits=15, random_state=46)
    preds, oof = trainer(_train, _test, cv, qda_params, model_type='qda', variance=True, _plot=False, _scaler=False)
    
    df = pd.DataFrame({
        'id': train['id'].values,
        'target': train['target'].values,
        'oof': oof
    })
    
    # oof, targetと乖離がある範囲を抽出
    df_0 = df[(df['target'] == 1) & (df['oof'] < 0.01)]
    df_1 = df[(df['target'] == 0) & (df['oof'] > 0.99)]
    
    modified_train.loc[df_0.index, 'target'] = 0
    modified_train.loc[df_1.index, 'target'] = 1
    modified_rows = len(df_0) + len(df_1)
    
    print('Modified {} rows'.format(modified_rows))
    
    return modified_train, modified_rows


def run_change_label(train, test, reg_param, _iter=20):
    reg_param = random.uniform(0.1, 0.6)
    for i in range(_iter):
        train, modified_rows = change_label(train, test, reg_param)
        train.reset_index(drop=True, inplace=True)
        if modified_rows < 10:
            break
    return train

In [ ]:
%%time
prep_start = time.time()

# PsuedoLabeling
train = run_PsuedoLabeling(train, test, reg_param=0.5, _min=0.04, _max=0.96, _iter=10)

# Change Effective Label
train = run_change_label(train, test, reg_param=0.5, _iter=3)

train.to_csv('Preprocessed_train.csv')

In [ ]:
prep_end = time.time()
elapsedtime = prep_end - prep_start
s = datetime.timedelta(seconds=elapsedtime)
print('Preprocessing 1 epoch time: {}'.format(str(s)))
print('Final Train Data Shape: {}'.format(train.shape))

# Model

In [ ]:
# oof, pred Data Container
tr = pd.DataFrame({
    'target': train['target'].values
})

te = pd.DataFrame()

**QDA**

In [ ]:
%%time
name = 'qda'
qda_params = {
    'reg_param':  0.01849,
    'priors': 17.0978,
    'tol': 6.943e-06
}

cv = StratifiedKFold(n_splits=15, random_state=46, shuffle=True)

preds, oof = trainer(train, test, cv, qda_params, model_type=name.split('_')[0], 
                     variance=True, _scaler=True, threshold=2.99)

tr[name] = oof
te[name] = preds

**NuSVC**

In [ ]:
%%time
name = 'nusvc'
nusvc_params = {
    'probability': True,
    'kernel': 'poly',
    'degree': 3,
    'gamma': 'auto',
    'coef0': 0.159,
    'nu': 0.1486,
    'tol': 1.8909e-4
}

cv = StratifiedKFold(n_splits=5, random_state=46)

preds, oof = trainer(train, test, cv, nusvc_params, model_type=name.split('_')[0], SVD=20)

tr[name] = oof
te[name] = preds

**GMM**

In [ ]:
%%time
name = 'gmm'
gmm_params = {
    'n_components': 2, 
    'init_params': 'kmeans', 
    'covariance_type': 'full', 
    'tol': 0.001,
    'reg_covar': 0.000389422,
    'max_iter': 536, 
    'n_init': 1
}

cv = StratifiedKFold(n_splits=5, random_state=46, shuffle=True)

preds, oof = trainer(train, test, cv, gmm_params, model_type=name.split('_')[0], 
                     variance=True, _scaler=False, threshold=2.21)

tr[name] = oof
te[name] = preds

# Stacking

In [ ]:
# create Rank
for col in te.columns:
    tr_rank = tr[col].rank(ascending=True)
    te_rank = te[col].rank(ascending=True)
    
    # ランクを正規化
    tr['{}_rank'.format(col)] = tr_rank.apply(lambda x: (x - tr_rank.min()) / (tr_rank.max() - tr_rank.min()))
    te['{}_rank'.format(col)] = te_rank.apply(lambda x: (x - te_rank.min()) / (te_rank.max() - te_rank.min()))

In [ ]:
def Stacking_trainer(tr, te, cols, clf, NAME, _scaler=False):
    _tr, _te = tr.copy(), te.copy()
    
    cv = StratifiedKFold(n_splits=5, random_state=46)
    X = _tr[cols].values
    Y = _tr['target'].values
    oof = np.zeros(len(tr)) 
    preds = np.zeros(len(te))
    _te = _te[cols]
    
    if _scaler:
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
        _te = scaler.transform(_te)

    for trn_index, val_index in cv.split(X, Y):
        X_trn, Y_trn = X[trn_index], Y[trn_index]
        X_val, Y_val = X[val_index], Y[val_index]

        clf.fit(X_trn, Y_trn)
        oof[val_index] = clf.predict_proba(X_val)[:,1]
        preds += clf.predict_proba(_te)[:,1] / cv.n_splits

    # Contain oof, pred data
    tr[NAME] = oof
    te[NAME] = preds
    
    # Plotting Histogram
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
    sns.distplot(tr[NAME], ax=axes[0])
    sns.distplot(te[NAME], ax=axes[1])
    axes[0].set_title('oof')
    axes[1].set_title('pred')
    plt.show()
    
    print(NAME)
    print('AUC: {:.5f}'.format(roc_auc_score(tr['target'].values, oof)))
    
    return tr, te

In [ ]:
tr_stage0 = tr.copy()
col_not_rank = [col for col in tr_stage0.columns if col not in ['target'] and 'rank' not in col]
col_in_rank = [col for col in tr_stage0.columns if col not in ['target'] and 'rank' in col]

In [ ]:
%%time
stack_stage1_dict = {
    'Stack_LR_1': LogisticRegression(penalty='l2', solver='liblinear', random_state=1),
    'Stack_LR_2': LogisticRegression(penalty='l2', solver='liblinear', random_state=10),
    'Stack_LR_3': LogisticRegression(penalty='l2', solver='liblinear', random_state=100),
    'Stack_LR_4': LogisticRegression(penalty='l2', solver='liblinear', random_state=1000) 
}

for name, clf in stack_stage1_dict.items():
    # Only use Rank
    tr, te = Stacking_trainer(tr, te, col_in_rank, clf, name, _scaler=False)

In [ ]:
tr['Stack_LR_Mean'] = (tr['Stack_LR_1'] + tr['Stack_LR_2'] + tr['Stack_LR_3'] + tr['Stack_LR_4']) / 4
te['Stack_LR_Mean'] = (te['Stack_LR_1'] + te['Stack_LR_2'] + te['Stack_LR_3'] + te['Stack_LR_4']) / 4

In [ ]:
plt.figure(figsize=(12, 8))
cols = [col for col in tr.columns if col not in ['target'] and '_rank' not in col]
sns.heatmap(te[cols].corr(), vmax=1.0, vmin=0.0, center=0.5, cmap='coolwarm', annot=True, fmt='.2f', square=True)
plt.title('Correlation Table by Model Predictions')
plt.show()

# Plotting

In [ ]:
def oof_distribution(tr, cols):
    ncols = 3
    nrows = int(np.ceil(len(cols) / 3))
    figsize = (12, 4 * nrows)
    fig, axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=figsize)
    
    for col, ax in zip(cols, axes.ravel()):
        
        auc = roc_auc_score(tr['target'], tr[col])
        
        tr_0 = tr[tr['target'] == 0]
        tr_1 = tr[tr['target'] == 1]
        
        tr_0 = tr_0[tr_0[col] > 0.0]
        tr_0 = tr_0[tr_0[col] < 1.0]
        tr_1 = tr_1[tr_1[col] > 0.0]
        tr_1 = tr_1[tr_1[col] < 1.0]
        
        tr_0 = tr_0.sample(5000)
        tr_1 = tr_1.sample(5000)
        
        sns.kdeplot(tr_0[col].values, ax=ax, shade=True, color="b", label='0')
        sns.kdeplot(tr_1[col].values, ax=ax, shade=True, color="r", label='1')
        ax.set_title('{}  AUC: {:.5f}'.format(col, auc))
        
    plt.tight_layout()
    plt.show()

In [ ]:
cols = [col for col in tr.columns if col not in ['target'] and '_rank' not in col]
oof_distribution(tr, cols)

# Submit

In [ ]:
submit = pd.read_csv('../input/sample_submission.csv')
submit['target'] = te['Stack_LR_Mean'].values
submit.to_csv('submission.csv', index=False)

In [ ]:
end = time.time()
elapsedtime = end - start
s = datetime.timedelta(seconds=elapsedtime)

print('This Kernel Runnnig Time: {}'.format(str(s)))